# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: Unveiling the Power of DeepLearning.AI

Embark on a journey into the realm of artificial intelligence with DeepLearning.AI. This online platform offers a gateway to understanding and mastering the complexities of deep learning. From beginners to experts, DeepLearning.AI provides courses curated by industry professionals like Andrew Ng. Dive into the world of neural networks, computer vision, and natural language processing. Elevate your skills, advance your career, and be at the forefront of technological innovation. Join a global community of learners and unravel the mysteries of AI with DeepLearning.AI. The future is now - are you ready to shape it?


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Embark on a journey into the realm of artificial intelligence with DeepLearning.AI. This online platform offers a gateway to understanding and mastering the complexities of deep learning. From beginners to experts, DeepLearning.AI provides courses curated by industry professionals like Andrew Ng. Dive into the world of neural networks, computer vision, and natural language processing. Elevate your skills, advance your career, and be at the forefront of technological innovation. Join a global community of learners and unravel the mysteries of AI with DeepLearning.AI. The future is now - are you ready to shape it?

------------------------------------------------------------------------

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Embark on a journey into the realm of artificial intelligence with DeepLearning.AI. This online platform offers a gateway to understanding and mastering the complexities of deep learning. From beginners to experts, DeepLearning.AI provides courses curated by industry professionals like Andrew Ng. Dive into the world of neural networks, computer vision, and natural language processing. Elevate your skills, advance your career, and be at the forefront of technological innovation. Join a global community of learners and unravel the mysteries of AI with DeepLearning.AI. The future is now - are you ready to shape it?

------------------------------------------------------------------------


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

Aggregating feedback from all SEO Reviewers, the following key points emerge:
- Incorporate relevant keywords like "artificial intelligence courses", "deep learning online platform", and "Andrew Ng" for enhanced search engine visibility.
- Add meta title and description tags with targeted keywords for better search engine optimization.
- Utilize heading tags (H1, H2, etc.) to structure content and highlight key topics such as neural networks, computer vision, and natural language processing for improved search rankings.

Final Suggestions:
1. Enhance the use of keywords throughout the content to improve SEO.
2. Implement meta title and description tags with targeted keywords for better optimization.
3. Properly structure content with heading tags to highlight key topics to further boost search rankings.

Overall, implementing these suggestions will likely improve the search eng

## Get the summary

In [15]:
print(res.summary)

Title: Mastering Artificial Intelligence Courses with DeepLearning.AI

Unlock the potential of artificial intelligence courses with DeepLearning.AI, a leading online platform founded by renowned expert Andrew Ng. Dive into neural networks, computer vision, and natural language processing through expert-led courses. Join a global community of learners to shape the future of technology. Whether you're a beginner or seasoned professional, DeepLearning.AI offers a pathway to advance your skills and career in AI. Discover the power of deep learning and stay ahead in this rapidly evolving field. Start your journey with DeepLearning.AI today.
